# Modify the user account

<div class="alert alert-block alert-info">
<i>Hello there.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>modify_user_account!!!</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    
</div>

In [ ]:
import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

#from get_resource_directory import get_resource_directory


<div class="alert alert-block alert-success">
    <p>This is our modify_ilo_user_account function overview, in the end this will give us about modifying the ilo user account related information.</p>
    <p>Let me give you a walk through of it!</p>
    <ol>
    <li> Users are having a facility to modify their ilo accounts. They can change their username, password and role using the below function.</li>
    <li> If we don't have a resource directory, information about the account is stored inside the <b>account_service</b> json object. Through <b>account_service_response</b> variable we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods">GET operation</a> to and trying to obtain the account details of an user in iLO system.</li>
        <li> <b>account_collection_uri</b> gives the response for that member about the <b>user information</b></li>
        <li> If the resource directory is present, then we will simply iterate through all instances inside resources and if the <b>account_collection_uri</b> is present in <b>@odata.type</b> attribute then we will store information in a dictionary called <b>body</b> which recieves the values like <b>new_username, new_password</b> and is updated.</li>
        <li> If a non-existing user tries to change the account, then it throws error.</li>
        <li> Finally, the <b>username</b> and <b>password</b> is updated.</li>
    </ol>
</div>

In [ ]:
def modify_ilo_user_account(_redfishobj, username_to_modify, new_loginname, new_username, \
                         new_password, role_id, privilege_dict):

    account_collection_uri = None

    resource_instances = _redfishobj.get_resource_directory()

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually to obtain
        #account info
        account_service_uri = _redfishobj.root.obj['AccountService']['@odata.id']
        account_service_response = _redfishobj.get(account_service_uri)
        account_collection_uri = account_service_response.obj['Accounts']['@odata.id']
        #modify role id
        if role_id:
            body = {"RoleId": role_id}
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#ManagerAccountCollection.' in instance['@odata.type']:
                account_collection_uri = instance['@odata.id']
        if privilege_dict:
            #HPE server, so modify privileges
            body = {"Oem": {"Hpe": {"Privileges": {}}}}
            for priv in privilege_dict:
                body["Oem"]["Hpe"]["Privileges"][priv] = privilege_dict[priv]
        if new_loginname:
            #modify login name
            body["Oem"]["Hpe"]["LoginName"] = new_loginname

    if new_username:
        body["UserName"] = new_username
    if new_password:
        body["Password"] = new_password

    #find the account to modify
    account_uri_to_modify = None
    account_uris = REDFISHOBJ.get(account_collection_uri)
    for account_uri in account_uris.dict['Members']:
        account = REDFISHOBJ.get(account_uri['@odata.id'])
        if account.dict['UserName'] == username_to_modify:
            account_uri_to_modify = account_uri['@odata.id']
            break

    if not account_uri_to_modify:
        sys.stderr.write("Cannot find account to modify")
        return

    #modify the account
    resp = REDFISHOBJ.patch(account_uri_to_modify, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success">
<b> <h1> 3.</h1>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE.<br>
    <ul>
        <li>Once you enter everything a redfish obejct is created with the help of RedfishClient class and a session is created and now you will  be logged in to the server</li>
        <li>If server is down or Unreachable we get "server not reachable" message!</li>
        <li>and Once everything is done you are automatically logged out with <b>.logout()<b> 😁. </li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = input("Enter iLO IP address: ")
    LOGIN_ACCOUNT = input("Enter Username: ")
    LOGIN_PASSWORD = getpass.getpass("Enter password: ")

    #username of the account to modify
    USERNAME_TO_MODIFY = input("Enter the username to be modified: ")
    # USERNAME_TO_MODIFY = "bruce_wayne"

    #account login name to change the account to
    NEW_LOGINNAME = input("Enter the new login name: ")
    # NEW_LOGINNAME = "joker"

    #account user name to change the account to
    NEW_USERNAME = input("Enter the new username: ")
    # NEW_USERNAME = "joker"

    #account password to change the account to
    NEW_PASSWORD = getpass.getpass("Enter the new password: ")
    # NEW_PASSWORD = "joker123"

    #role to change account to
    ROLE_ID = input("Enter the Role ID: (Administrator, ReadOnly or Operator are available)")
    # ROLE_ID = "ReadOnly"

    #update HPE account privileges
    PRIVILEGE_DICT = {"iLOConfigPriv": False, "VirtualMediaPriv": False, "RemoteConsolePriv": True,\
                      "UserConfigPriv": False, "VirtualPowerAndResetPriv": False, \
                      "SystemRecoveryConfigPriv": False, "LoginPriv": True, \
                      "HostStorageConfigPriv": False, "HostNICConfigPriv": False, \
                      "HostBIOSConfigPriv": False}
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    modify_ilo_user_account(REDFISHOBJ, USERNAME_TO_MODIFY, NEW_LOGINNAME, NEW_USERNAME, \
                            NEW_PASSWORD, ROLE_ID, PRIVILEGE_DICT)

    REDFISHOBJ.logout()


if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    find_ilo_mac_address(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class="alert alert-block alert-success">
    In case you need help 🔗 links to relevant resources :<br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
</div>